## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-16-54-28 +0000,bbc,AWS outage: Is the world relying too much on U...,https://www.bbc.com/news/articles/c0jdgp6n45po...
1,2025-10-21-16-49-01 +0000,bbc,Boris Johnson left the Covid Inquiry in a hurr...,https://www.bbc.com/news/articles/cj6nennz0nyo...
2,2025-10-21-16-48-02 +0000,startribune,RandBall: Vikings fans are getting antsy to se...,https://www.startribune.com/minnesota-vikings-...
3,2025-10-21-16-48-00 +0000,wsj,GM Shares Surge 15% on Raised Guidance,https://www.wsj.com/business/autos/gm-earnings...
4,2025-10-21-16-46-01 +0000,nypost,Long Island ‘candy man’ busted after middle-sc...,https://nypost.com/2025/10/21/us-news/long-isl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,52
122,new,16
36,up,15
189,gaza,13
151,house,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2025-10-20-22-36-00 +0000,wsj,"President Trump’s $100,000 H-1B visa fee will ...",https://www.wsj.com/politics/policy/u-s-narrow...,130
306,2025-10-20-20-06-32 +0000,latimes,Trump again suggests sending troops to San Fra...,https://www.latimes.com/california/story/2025-...,123
56,2025-10-21-14-55-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...,122
315,2025-10-20-19-28-34 +0000,nypost,Demolition begins on East Wing of White House ...,https://nypost.com/2025/10/20/us-news/demoliti...,113
246,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,130,2025-10-20-22-36-00 +0000,wsj,"President Trump’s $100,000 H-1B visa fee will ...",https://www.wsj.com/politics/policy/u-s-narrow...
68,80,2025-10-21-14-19-48 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
56,61,2025-10-21-14-55-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...
246,53,2025-10-20-23-05-53 +0000,nypost,"Trump can deploy National Guard to Portland, O...",https://nypost.com/2025/10/20/us-news/9th-circ...
203,50,2025-10-21-04-53-16 +0000,wapo,"Japan elects first female leader, a hawk who p...",https://www.washingtonpost.com/world/2025/10/2...
251,45,2025-10-20-23-00-00 +0000,wsj,Business and political leaders have intensifie...,https://www.wsj.com/politics/elections/nyc-may...
91,37,2025-10-21-13-25-26 +0000,nypost,Former French President Sarkozy begins 5-year ...,https://nypost.com/2025/10/21/world-news/forme...
81,35,2025-10-21-13-56-46 +0000,nypost,Warner Bros. Discovery says it’s open to a sal...,https://nypost.com/2025/10/21/business/warner-...
254,33,2025-10-20-22-46-37 +0000,nyt,"To Counter China’s Hold on Rare Minerals, Trum...",https://www.nytimes.com/2025/10/20/us/politics...
127,33,2025-10-21-11-17-00 +0000,wsj,The Treasury Department instructed employees n...,https://www.wsj.com/politics/policy/treasury-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
